In [1]:
_RIOT_API_KEY='RGAPI-79bd2f52-e028-4e9b-9a04-1764b72aecd8'
from riotwatcher import LolWatcher
import helper_functions as help
import constants
import pandas as pd
from datetime import datetime
lol_watcher = LolWatcher(_RIOT_API_KEY)


1. get puuid for a summoner name - Summonerv4/by-name/summonername
2. get list of match ids by puuid - Matchv5/ids
3. Check database for which match ids i don't have - find_to_slurp
4. Create a function that does the following
- get match by match id - Matchv5/matchids
    - Store in a DB
- get match timeline by match id
    - store in a DB
5. Create concurrent processing that will speed this up

In [2]:
match_list=help.get_matchlist_by_summoner_name('souporsecret', lol_watcher)
len(match_list)
arena_match = lol_watcher.match.by_id('na1', match_list[0])
flex_match = lol_watcher.match.by_id('na1', match_list[4])


In [3]:
queues_dict[440]

NameError: name 'queues_dict' is not defined

In [5]:
item_dict = constants.get_items(lol_watcher)
spell_dict = constants.get_spells(lol_watcher)
queues_dict = constants.get_queues()
match_info = flex_match['info']


#remove the challenge and perk info, we don't care about that shit. keep everything else
[match_info['participants'][i].pop('perks') for i in range(len(match_info['participants'])) ]
[match_info['participants'][i].pop('challenges') for i in range(len(match_info['participants'])) ]

#Turn the data into a DF format
df = pd.DataFrame(data=match_info['participants'], columns = list(match_info['participants'][0].keys()))
df_length = len(df)

# Add match_specific_data to each participant's row
df['matchId'] = [flex_match['metadata']['matchId'] for i in range(df_length)]
df['region'] = [flex_match['info']['platformId'] for i in range(df_length)]
df['gameDurationSeconds'] = [flex_match['info']['gameDuration'] for i in range(df_length)]
match_time = datetime.fromtimestamp(int(flex_match['info']['gameCreation']/1000))
df['gameCreationDate'] = [match_time for i in range(df_length)]
df['queueId'] = [flex_match['info']['queueId'] for i in range(df_length)]


# convert the Ids to Strings
df['winFlag'] =  df['win'].apply(str).map({'True':1, 'False':0})
df['teamName'] = df['teamId'].apply(str).map({'100':'Blue', '200':'Red'})
df['item1Name'] = df['item1'].apply(str).map(item_dict)
df['item2Name'] = df['item2'].apply(str).map(item_dict)
df['item3Name'] = df['item3'].apply(str).map(item_dict)
df['item4Name'] = df['item4'].apply(str).map(item_dict)
df['item5Name'] = df['item5'].apply(str).map(item_dict)
df['item6Name'] = df['item6'].apply(str).map(item_dict)
df['summoner1Name'] = df['summoner1Id'].apply(str).map(spell_dict)
df['summoner2Name'] = df['summoner2Id'].apply(str).map(spell_dict)
df['queueName'] = df['queueId'].apply(str).map(queues_dict)







In [6]:
df.to_csv('teehee.csv')


In [35]:

match_time = datetime.fromtimestamp(int(flex_match['info']['gameCreation']/1000))
print('match time: ', match_time)
print('match duration:', flex_match['info']['gameDuration'], 'seconds' )

match time:  2023-07-17 20:53:05
match duration: 2220 seconds


In [37]:
print(match_time)

2023-07-17 20:53:05


'2023-07-17 20:53:05'